# Homework 5

## Question 1: Install Spark and PySpark

- Install Spark
- Run PySpark
- Create a local spark session
- Execute spark.version.

What's the output?

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/10 11:29:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.version

'3.3.2'

## Question 2: Yellow October 2024

Read the October 2024 Yellow into a Spark Dataframe.

Repartition the Dataframe to 4 partitions and save it to parquet.

What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.

- 6MB
- 25MB
- 75MB
- 100MB

In [3]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-10 11:17:44--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.162.78.51, 3.162.78.164, 3.162.78.186, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.162.78.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M   106MB/s    in 0.6s    

2025-03-10 11:17:44 (106 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [4]:
df = spark.read \
    .option("header", "true") \
    .parquet('yellow_tripdata_2024-10.parquet') \
    .repartition(4)

df.write.parquet('yellow/2024/10/')

## Question 3: Count records 

How many taxi trips were there on the 15th of October?

Consider only trips that started on the 15th of October.

- 85,567
- 105,567
- 125,567
- 145,567

In [2]:
df_yellow = spark.read.parquet('yellow/*/*')

In [6]:
df_yellow.createOrReplaceTempView('trips_data')

In [12]:
spark.sql("""
SELECT
    count(1)
FROM
    trips_data
WHERE
    CAST(tpep_pickup_datetime AS DATE) = '2024-10-15'
    AND trip_distance > 0
""").show()

+--------+
|count(1)|
+--------+
|  126106|
+--------+



## Question 4: Longest trip

What is the length of the longest trip in the dataset in hours?

- 122
- 142
- 162
- 182

In [21]:
spark.sql("""
SELECT
    timestampdiff(HOUR, tpep_pickup_datetime, tpep_dropoff_datetime)
FROM
    trips_data
WHERE
    trip_distance > 0
ORDER BY
    timestampdiff(HOUR, tpep_pickup_datetime, tpep_dropoff_datetime) DESC
""").show()

+----------------------------------------------------------------+
|timestampdiff(HOUR, tpep_pickup_datetime, tpep_dropoff_datetime)|
+----------------------------------------------------------------+
|                                                             162|
|                                                             114|
|                                                              89|
|                                                              89|
|                                                              70|
|                                                              67|
|                                                              66|
|                                                              42|
|                                                              38|
|                                                              26|
|                                                              25|
|                                                             

## Question 6: Least frequent pickup location zone

Load the zone lookup data into a temp view in Spark:

```bash
wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
```

Using the zone lookup data and the Yellow October 2024 data, what is the name of the LEAST frequent pickup location Zone?

- Governor's Island/Ellis Island/Liberty Island
- Arden Heights
- Rikers Island
- Jamaica Bay

In [23]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-03-10 11:59:17--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.161.178.192, 3.161.178.162, 3.161.178.24, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.161.178.192|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0.001s  

2025-03-10 11:59:18 (9.04 MB/s) - ‘taxi_zone_lookup.csv’ saved [12331/12331]



In [25]:
df = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup.csv')

In [26]:
df.createOrReplaceTempView('taxi_zones')

In [27]:
spark.sql("""
SELECT
    tz.Zone,
    COUNT(1)
FROM
    trips_data td
    INNER JOIN taxi_zones tz
    ON td.PULocationID = tz.LocationID
WHERE
    trip_distance > 0
GROUP BY
    tz.Zone
ORDER BY
    COUNT(1)
""").show()

+--------------------+--------+
|                Zone|count(1)|
+--------------------+--------+
|Governor's Island...|       1|
|   Rossville/Woodrow|       1|
|       Rikers Island|       2|
|       Port Richmond|       2|
| Green-Wood Cemetery|       2|
|Charleston/Totten...|       3|
|       West Brighton|       3|
|         Jamaica Bay|       3|
|Eltingville/Annad...|       4|
|         Great Kills|       5|
|        Crotona Park|       6|
|     Mariners Harbor|       7|
|Heartland Village...|       7|
|             Oakwood|       8|
|       Broad Channel|       9|
|New Dorp/Midland ...|       9|
|Saint George/New ...|       9|
|         Westerleigh|      10|
|Breezy Point/Fort...|      10|
|     Pelham Bay Park|      11|
+--------------------+--------+
only showing top 20 rows

